In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pylab as plt 
import matplotlib as mpl
#for making sure matplotlib plots are generated in Jupyter notebook itself
%matplotlib inline
import geopandas as gpd
import random
import itertools
from shapely.validation import explain_validity
from shapely.geometry import MultiPolygon, Polygon, LineString, Point
from shapely.validation import make_valid
from libpysal.weights import Queen, Rook, KNN

In [ ]:
gdf=gpd.read_file(r'fittedgrid.shp')


In [ ]:
gdf.plot(figsize=(30,15), color='linen',alpha=0.5, edgecolor="k")

In [ ]:
gdf.head()

In [ ]:
len(gdf['id'].unique())

In [ ]:
gdf.columns

In [ ]:
gdf=gdf.drop(columns=['left', 'top', 'right', 'bottom'])

In [ ]:
gdf.head()

In [ ]:
gdf['Area']=gdf['geometry'].area/10**6
gdf['Perimeter']=gdf['geometry'].length/1000

In [ ]:
s = 150 
a = 3.00 
g = 0.035
b=np.log(a/((2*g*s)-1))-4*g*s + 4*g
K=np.log(5)/(1.5 - 1)

In [ ]:
#b=1.03573227355399
#K=3.2188758248682006
gdf['Military_power']=gdf['Area']/(a+np.exp(g*gdf['Perimeter']+b))
gdf['kingdom']=gdf['id']

In [ ]:
gdf.head()

In [ ]:
def neigbours(gdf,row):
    neighbours=w_rook.neighbors[row]
    if len(neighbours)==0:
        print('no battle')
        return    
    
    return neighbours

In [ ]:
def battle(gdf):

    attackernumber=random.choice(gdf.index.values)
    gdf['neighbours']=gdf['geometry'].touches(gdf.iloc[attackernumber].geometry)
    gdf.loc[gdf['neighbours']==True]
    power_ranked=gdf.loc[gdf['neighbours']==True].sort_values(by=['Military_power']).index
    if power_ranked.size==0:
        print('no neighbours, no battle')
        result=gdf
        return result
    targetnumber=power_ranked[0]
    print(gdf.at[attackernumber,'kingdom'],gdf.at[targetnumber,'kingdom'])
    if (gdf.loc[[attackernumber]].kingdom.values == gdf.loc[[targetnumber]].kingdom.values):
        print('no self battle')
        result=gdf
        return result
    poweroftarget=(gdf.at[targetnumber, 'Military_power'])
    powerofaggressor=(gdf.at[attackernumber, 'Military_power'])
    probofsuccess=1-0.5*np.exp(-K*(powerofaggressor/poweroftarget)-1)
    rand=random.uniform(0, 1)
    if rand > probofsuccess:
        (attackernumber,targetnumber)=(targetnumber,attackernumber)
    gdf.at[targetnumber,'kingdom']=gdf.at[attackernumber,'kingdom'] 
    row1=gdf.iloc[[targetnumber]] #target
    row2=gdf.iloc[[attackernumber]] #aggressor
    joined = pd.concat([row1.geometry, row2.geometry])
    polygon = joined.geometry.unary_union   
    gdf.at[attackernumber,'geometry']=polygon
    gdf.at[attackernumber,'Area']=gdf.at[attackernumber,'geometry'].area/10**6
    gdf.at[attackernumber,'Perimeter']=gdf.at[attackernumber,'geometry'].length/1000
    gdf.at[attackernumber,'Military_power']=gdf.at[attackernumber,'Area']/(a+np.exp(g*gdf.at[attackernumber,'Perimeter']+b))   
    ax=gdf.plot(figsize=(30,15), color='linen',alpha=0.5, edgecolor="k")
    (gdf.loc[[attackernumber],'geometry']).plot(ax=ax,alpha=0.5, edgecolor="k", color='forestgreen')    
    gdf=(gdf.drop(gdf.index[[targetnumber]]))
    gdf.reset_index(drop=True, inplace=True)
      
    result=gdf
    return result
        
    
    
    

In [ ]:
result=battle(gdf)

In [ ]:
#run N interations
results=[]
count=1
while count < 2000:
    print(count)
    result=battle(result)
    results.append(result)
    # Save the dataframe to a CSV file
    filename= ('battle1_%s.csv' % (count))
    result.to_csv(filename, index=False)
    count=count+1

In [ ]:
for i in range(0,1997,10):
    sorted=results[i].sort_values('Area',ascending=False)
    sorted.reset_index(drop=True, inplace=True)

    ax=gdf.plot(figsize=(30,15) , edgecolor="lightblue",color='linen', alpha=1)
    ax.set_facecolor('lightblue')
    sorted.loc[[0]].geometry.plot(ax=ax,alpha=0.5, edgecolor="lightblue", color='red')
    sorted.loc[[1]].geometry.plot(ax=ax,alpha=0.5, edgecolor="lightblue", color='rosybrown')
    sorted.loc[[2]].geometry.plot(ax=ax,alpha=0.5, edgecolor="lightblue", color='orange')
    sorted.loc[[3]].geometry.plot(ax=ax,alpha=0.5, edgecolor="lightblue", color='grey')
    sorted.loc[[4]].geometry.plot(ax=ax,alpha=0.5, edgecolor="lightblue", color='darkgrey')
    sorted.loc[[5]].geometry.plot(ax=ax,alpha=0.5, edgecolor="lightblue", color='crimson')
    sorted.loc[[6]].geometry.plot(ax=ax,alpha=0.5, edgecolor="lightblue", color='azure')
    sorted.loc[[7]].geometry.plot(ax=ax,alpha=0.5, edgecolor="lightblue", color='green')
    sorted.loc[[8]].geometry.plot(ax=ax,alpha=0.5, edgecolor="lightblue", color='blue')
    sorted.loc[[9]].geometry.plot(ax=ax,alpha=0.5, edgecolor="lightblue", color='darkblue') 

In [ ]:
patches=[]

for i in range(0,len(results)):
    patches.append(len(results[i]))
   # print(len(results[i]))

In [ ]:
x = np.linspace(0, len(results), len(results))
y=patches
# plot
fig, ax = plt.subplots()
ax.plot(x, y, linewidth=2.0)
plt.show()

In [ ]:
sizes=[]
for i in range(0,len(results)):
    sizes.append(max(results[i].Area))
    #print(max(results[i].Area))

In [ ]:
x = np.linspace(0, len(results), len(results))
y=sizes
# plot
fig, ax = plt.subplots()

ax.plot(x, y, linewidth=2.0)
plt.show()

In [ ]:
avesizes=[]
for i in range(0,len(results)):
    avesizes.append(max(results[i].Area)/len(results[i]))
    #print(max(results[i].Area))

In [ ]:
x = np.linspace(0, len(results), len(results))
y=avesizes
# plot
fig, ax = plt.subplots()

ax.plot(x, y, linewidth=2.0)
plt.xlim([1740, 1753]) 
plt.show()

In [ ]:
results=results[0:1753]

kingdoms=[]
for i in range(0,len(results)):
    kingdoms.append(len(results[i].kingdom.unique()))
    #print(max(results[i].Area))

In [ ]:
x = np.linspace(0, len(results), len(results))
y=kingdoms
# plot
fig, ax = plt.subplots()

ax.plot(x, y, linewidth=2.0)
#plt.xlim([1727, 1800]) 
#plt.ylim([0, 250]) 
plt.show()

In [ ]:


#dy/dt=cAexp (√A/h)-a

In [ ]:
sizes=[]
for i in range(0,len(results)):
    sizes.append(max(results[i].Area))
    #print(max(results[i].Area))



x = np.linspace(0, len(results), len(results))
y=sizes
# plot
fig, ax = plt.subplots()

ax.plot(x, y, linewidth=2.0)
plt.show()

In [ ]:
#dy/dt=cAexp (√A/h)-a


def model_f(x,a,b,c):
  return a*np.exp(b * np.sqrt(x) ) -c

In [ ]:
popt, pcov = curve_fit(model_f, x, y, maxfev = 80000000)

In [ ]:
popt

In [ ]:
popt[0]

In [ ]:
a=popt[0]
b=popt[1]
c=popt[2]



x=np.linspace(0,1753,1753)
x1 = np.linspace(0,1753,1753)
y1=a*np.exp(b * np.sqrt(x) ) -c

ax=plt.plot(x,y, label='Simulated curve')
plt.plot(x,y1, marker='o', markevery=100, label='Fitted curve')


plt.xlabel("iterations")
plt.ylabel("Area")
plt.legend(loc="upper left")


